# Decision Tree and Random Forest(2)

***

## Breast Cancer in [cost complexity pruning](https://scikit-learn.org/stable/modules/tree.html#tree-algorithms) technique of DecisionTreeClassifier

<b>Load and return the Breast Cancer dataset</b> : [for the further information about the dataset](https://github.com/HaeHwan/dataset/blob/master/Breast%20Cancer%20Wisconsin/Breast%20Cancer%20Wisconsin.ipynb)

In [1]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.22.1.


> 1. Run Anaconda Prompt as adminstrator
> 2. conda install scikit-learn=0.22.1

In [2]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
from sklearn.tree import DecisionTreeClassifier

In [3]:
X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

train_test_split : default = 0.25

In [4]:
X.shape, X_train.shape, X_test.shape

((569, 30), (426, 30), (143, 30))

In [5]:
clf = DecisionTreeClassifier(random_state=0)
clf.cost_complexity_pruning_path(X_train, y_train)

{'ccp_alphas': array([0.        , 0.00226647, 0.00464743, 0.0046598 , 0.0056338 ,
        0.00704225, 0.00784194, 0.00911402, 0.01144366, 0.018988  ,
        0.02314163, 0.03422475, 0.32729844]),
 'impurities': array([0.        , 0.00453294, 0.01847522, 0.02313502, 0.02876883,
        0.03581108, 0.04365302, 0.05276704, 0.0642107 , 0.0831987 ,
        0.10634033, 0.14056508, 0.46786352])}